In [1]:
#This is a variant of "taketwo-dsmvp-v1" for the Jigsaw data.

In [2]:
#It is also a variant of the version that was pushed to the main branch of TakeTwo repo in Feb 21. 

In [3]:
#This is a version of "v1" demo notebook.

In [4]:
#CODE DEPENDENCY: sklearn, numpy, pandas, random, string. It uses from sklearn NB (Naive Bayes) and SGD (Support Vector Machine)

In [5]:
#DATA DEPENDENCY: In this version the "Jigsaw unintended biased in toxicity" data is used for demo purpose. 

In [6]:
#See https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data?select=train.csv

In [7]:
#for detailed description of the data and instructions to download them. 

In [8]:
#As of now we are using only the "train.csv" portion of the data, which needs to be in the calling directory. 

In [9]:
from sklearn.datasets import fetch_20newsgroups

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
from sklearn.linear_model import SGDClassifier

In [12]:
from sklearn.naive_bayes import MultinomialNB

In [13]:
from sklearn.pipeline import Pipeline

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
import numpy as np

In [17]:
import pandas as pd

In [18]:
import random

In [19]:
import string

In [20]:
#Fetch the Jigsaw data and store them in appropriate arrays

In [21]:
jigsawTrain = pd.read_csv('train.csv')

In [22]:
jigsawTrain.shape

(1804874, 45)

In [23]:
allTrain = pd.DataFrame(jigsawTrain, columns = ['id', 'comment_text', 'severe_toxicity'])

In [24]:
allTrain.shape

(1804874, 3)

In [25]:
allTrain1M = allTrain.head(1000000)

In [26]:
allTrain1M.shape

(1000000, 3)

In [27]:
#allTrain = allTrain1M

In [28]:
blackData = jigsawTrain[(jigsawTrain.black == 1.0)]

In [29]:
allTrain.shape

(1804874, 3)

In [30]:
blackTrain = pd.DataFrame(blackData, columns = ['id', 'comment_text', 'severe_toxicity', 'black'])

In [31]:
blackTrain.shape

(7656, 4)

In [32]:
blackTrainId = blackTrain['id']

In [33]:
blackTrainText = blackTrain['comment_text']

In [34]:
blackTrainSevereToxicity = blackTrain.loc[:,'severe_toxicity']

In [35]:
blackTrainText.head(1)

516    BJ, your suggestion that low income residents ...
Name: comment_text, dtype: object

In [36]:
allTrainId = allTrain['id']

In [37]:
allTrainText = allTrain['comment_text']

In [38]:
allTrainSevereToxicity = allTrain.loc[:,'severe_toxicity']

In [39]:
allTrainSevereToxicity.head()

0    0.000000
1    0.000000
2    0.000000
3    0.000000
4    0.021277
Name: severe_toxicity, dtype: float64

In [40]:
blackTrain.head(5)

,id,comment_text,severe_toxicity,black
516,240627,"BJ, your suggestion that low income residents ...",0.0,1.0
1428,242346,"Well, this wasn't meant to be a dissertation. ...",0.0,1.0
1527,242454,So Hillary's appeal is to Democrats who make o...,0.0,1.0
1569,242498,Is the real point of this article to show that...,0.0,1.0
1623,242562,> A less sensational headline would be “Black ...,0.0,1.0


In [41]:
allTrain.head(5)

,id,comment_text,severe_toxicity
0,59848,"This is so cool. It's like, 'would you want yo...",0.000000
1,59849,Thank you!! This would make my life a lot less...,0.000000
2,59852,This is such an urgent design problem; kudos t...,0.000000
3,59855,Is this something I'll be able to install on m...,0.000000
4,59856,haha you guys are a bunch of losers.,0.021277


In [42]:
blackTrainId.iloc[0]

240627

In [43]:
blackTrainText.iloc[0]

"BJ, your suggestion that low income residents of Portland do not understand their own needs and desires perplexes me.  It is a bit snobby- as if the poor need middle class whites to tell them what they should want. And, for blacks and latinos, home ownership[ is pretty much their only chance for a nest egg.  Unless they are union they don't really get the jobs with stock options and generous 401k matches.  The fact that redlining once existed doesn't make the handful of black residents around Alberta less attached to their homes.\n\nYou are struggling with he fact that the working class folks in Portland do not want to live the New Urbanist dream.  Look at the big study Metro did to gauge housing preferences. \n\nFinally, I note you put a lot of trust in the people who own large apartment complexes to not exploit the inherent vulnerability that comes with being a renter.  The SFH gives people autonomy.  Yes, housing prices can drop, but so can any other investment.  Take my 401k- plea

In [44]:
blackTrainSevereToxicity.iloc[0]

0.0

In [45]:
len(blackTrain)

7656

In [46]:
categories = ["non-toxic", "toxic"]

In [47]:
from collections import namedtuple
data_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [48]:
def flipCoin(bias):
    f = random.random()
    return True if f<bias else False

In [49]:
for i in range(0,10):
    print(flipCoin(0.25))

True
True
True
False
False
True
False
False
True
True


In [50]:
for x in range(len(blackTrain)):
    if (blackTrainSevereToxicity.iloc[x] > 0):
        data_set.target.append(1)
        data_set.urls.append(blackTrainId.iloc[x])
        data_set.data.append(blackTrainText.iloc[x])
    elif flipCoin(0.25):
        data_set.target.append(0)
        data_set.urls.append(blackTrainId.iloc[x])
        data_set.data.append(blackTrainText.iloc[x])

In [51]:
from collections import namedtuple
all_data_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [52]:
from collections import namedtuple
all_train_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [53]:
from collections import namedtuple
all_test_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [54]:
downSampling = 0.06

In [55]:
for x in range(len(allTrain)):
    if (allTrainSevereToxicity.iloc[x] > 0):
        all_data_set.target.append(1)
        all_data_set.urls.append(allTrainId.iloc[x])
        all_data_set.data.append(allTrainText.iloc[x])
    elif flipCoin(downSampling):
        all_data_set.target.append(0)
        all_data_set.urls.append(allTrainId.iloc[x])
        all_data_set.data.append(allTrainText.iloc[x])

In [56]:
#Here is a version in which training and test data are separately generated from all data

In [57]:
trainRatio = 0.9

In [58]:
for x in range(len(allTrain)):
    if (allTrainSevereToxicity.iloc[x] > 0):
        if flipCoin(trainRatio): 
            all_train_set.target.append(1)
            all_train_set.urls.append(allTrainId.iloc[x])
            all_train_set.data.append(allTrainText.iloc[x])
        else: 
            all_test_set.target.append(1)
            all_test_set.urls.append(allTrainId.iloc[x])
            all_test_set.data.append(allTrainText.iloc[x])
    elif flipCoin(downSampling):
        if flipCoin(trainRatio): 
            all_train_set.target.append(0)
            all_train_set.urls.append(allTrainId.iloc[x])
            all_train_set.data.append(allTrainText.iloc[x])
        else: 
            all_test_set.target.append(0)
            all_test_set.urls.append(allTrainId.iloc[x])
            all_test_set.data.append(allTrainText.iloc[x])

In [59]:
#either leave data_set as is (generated from blackData) or set it to all_data_set (generated from all the data)

In [60]:
data_set = all_test_set

In [61]:
len(data_set.target)

20630

In [62]:
 data_set.target[0:10]

[1, 1, 1, 0, 1, 0, 0, 0, 1, 0]

In [63]:
sum(data_set.target)

10275

In [64]:
data_set.urls[0:10]

[59856, 59859, 239612, 239721, 239984, 240542, 240587, 240897, 241346, 241632]

In [65]:
data_set.data[0:2]

['haha you guys are a bunch of losers.', 'ur a sh*tty comment.']

In [66]:
marker_train = all_train_set

In [67]:
#Here a separate test data set should be used.

In [68]:
marker_test = all_test_set

In [69]:
for t in marker_train.target[:10]:
...     print(marker_train.target_names[t])

toxic
non-toxic
toxic
non-toxic
non-toxic
toxic
non-toxic
non-toxic
non-toxic
non-toxic


In [70]:
#marker_train.data is an array of strings (containing as many "texts" as there are coming from Marker.)

In [71]:
#marker_train.target is an array of integers (containing the class indices for the above texts' class labels)

In [72]:
#marker_train_target_names is an array of class label names indexed by the array indices.

In [73]:
len(marker_train.target)

186556

In [74]:
sum(marker_train.target)

94361

In [75]:
len(marker_test.data)

20630

In [76]:
for t in marker_train.target[:10]:
...     print(marker_train.target_names[t])

toxic
non-toxic
toxic
non-toxic
non-toxic
toxic
non-toxic
non-toxic
non-toxic
non-toxic


In [77]:
for t in marker_test.target[:10]:
...     print(marker_train.target_names[t])

toxic
toxic
toxic
non-toxic
toxic
non-toxic
non-toxic
non-toxic
toxic
non-toxic


In [78]:
#Classes for model representations packaging the components in the modeling pipeline.
#For SGDClassifier and NaiveBayes.
#We may want to extend it to utilize "Pipeline()" as in "ModelRepSGD0", which is not in use at the moment.

In [79]:
class ModelRepSGD:
    def __init__(self, count_vect, X_train_counts, tf_transformer, text_clf):
        self.count_vect = CountVectorizer()
        self.X_train_counts = self.count_vect.fit_transform(marker_train.data)
        self.tf_transformer = TfidfTransformer(use_idf=False).fit(self.X_train_counts)
        self.text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                  alpha=1e-3, random_state=42,
                                  max_iter=5, tol=None)),
        ])

In [80]:
class ModelRepNB:
    def __init__(self, count_vect, X_train_counts, tf_transformer, clf):
        self.count_vect = CountVectorizer()
        self.X_train_counts = self.count_vect.fit_transform(marker_train.data)
        self.tf_transformer = TfidfTransformer(use_idf=False).fit(self.X_train_counts)
        self.clf = MultinomialNB()

In [81]:
#Functions for model training - for SGDClassifier and NaiveBayes (SGD one is untested)

In [82]:
def model_train_SGD(marker_train): 
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(marker_train.data)
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                              alpha=1e-3, random_state=42,
                              max_iter=5, tol=None)),
    ])
    text_clf.fit(marker_train.data, marker_train.target)
    trained_model = ModelRepSGD(count_vect, X_train_counts, tf_transformer, text_clf)
    trained_model.text_clf = text_clf
    return trained_model

In [83]:
def model_train_NB(marker_train): 
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(marker_train.data)
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)
    clf = MultinomialNB().fit(X_train_tf, marker_train.target)
    trained_model = ModelRepNB(count_vect, X_train_counts, tf_transformer, clf)
    trained_model.clf = clf
    return trained_model

In [84]:
#Function for model testing (common for SGDClassifier and NB)

In [85]:
def model_test(flagger_new_test, trained_model):
    count_vect = trained_model.count_vect
    tf_transformer = trained_model.tf_transformer
    clf = trained_model.clf
    X_new_counts = count_vect.transform(flagger_new_test)
    X_new_tfidf = tf_transformer.transform(X_new_counts)
    predicted = clf.predict(X_new_tfidf)
    return predicted

In [86]:
#Now testing model_train and model_test

In [87]:
trained_model_NB = model_train_NB(marker_train)

In [88]:
trained_model_SGD = model_train_SGD(marker_train)

In [89]:
#Inspecting the components of the trained model

In [90]:
type(trained_model_NB)

__main__.ModelRepNB

In [91]:
trained_model_NB.count_vect

CountVectorizer()

In [92]:
trained_model_NB.tf_transformer

TfidfTransformer(use_idf=False)

In [93]:
trained_model_NB.clf

MultinomialNB()

In [94]:
trained_model_SGD.count_vect

CountVectorizer()

In [95]:
trained_model_SGD.tf_transformer

TfidfTransformer(use_idf=False)

In [96]:
trained_model_SGD.text_clf

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [97]:
trained_model_SGD.count_vect

CountVectorizer()

In [98]:
trained_model_SGD.tf_transformer

TfidfTransformer(use_idf=False)

In [99]:
#Calling model tester/predictor on a single new text from Flagger

In [100]:
#flagger_new_text = fetch_20newsgroups(subset='test',
#                                      categories=categories, shuffle=True, random_state=42)

In [101]:
flagger_new_text = ['haha you guys are a bunch of losers.']

In [102]:
predicted_NB = model_test(flagger_new_text, trained_model_NB)

In [103]:
predicted_SGD = trained_model_SGD.text_clf.predict(flagger_new_text)

In [104]:
predicted_NB

array([1])

In [105]:
predicted_SGD

array([1])

In [106]:
#Now evaluating NB and SGD models on a test data set

In [107]:
predicted_test_NB = model_test(marker_test.data, trained_model_NB)

In [108]:
predicted_test_SGD = trained_model_SGD.text_clf.predict(marker_test.data)

In [109]:
predicted_test_SGD.shape

(20630,)

In [110]:
sum(predicted_test_SGD)

8760

In [111]:
sum(predicted_test_NB)

10969

In [112]:
predicted_test_SGD[0:20]

array([1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])

In [113]:
np.mean(predicted_test_NB == marker_test.target)

0.7572467280659234

In [136]:
np.mean(predicted_test_SGD == marker_test.target)

0.7133190536461883

In [ ]:
!pip install -U ibm_watson_machine_learning
!pip install -U ibm-cos-sdk-core


In [139]:
from ibm_watson_machine_learning import APIClient

In [140]:
# add apikey here
wml_credentials = {
                   "url": "https://eu-gb.ml.cloud.ibm.com",
                   "apikey":"YkU_2XKSqz_ujIoL1sAU-R3jniWTTVu3U4jhWpHsW_nm"
                  }

client = APIClient(wml_credentials)

In [141]:
space_metadata = {
    'name': 'taketwo-wml-space',
    'description': 'taketwo-wml-space',
}

In [142]:
#obtained this from WML resource on IBM Cloud
space_id = '2d3e8ea7-efa2-4efd-ae26-8a46e7a714df'

In [143]:
client.spaces.list(limit=10)

------------------------------------  -----------------  ------------------------
ID                                    NAME               CREATED
2d3e8ea7-efa2-4efd-ae26-8a46e7a714df  taketwo-wml-space  2021-07-16T00:54:38.389Z
------------------------------------  -----------------  ------------------------


In [144]:
client.set.default_space(space_id)

'SUCCESS'

In [145]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.7")

In [283]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'Take Two Scikit model',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

# commenting out for now
# published_model = client.repository.store_model(
#     model=trained_model_NB.clf,
#     meta_props=metadata,
#     training_data=None,
#     training_target=None)


In [10]:
# import json

# published_model_uid = client.repository.get_model_uid(published_model)
# model_details = client.repository.get_details(published_model_uid)
# print(json.dumps(model_details, indent=2))

NameError: name 'published_model' is not defined

In [173]:
models_details = client.repository.list_models()

------------------------------------  ---------------------  ------------------------  -----------------
ID                                    NAME                   CREATED                   TYPE
d195ce8b-182f-44e1-a424-817bef2896a4  Take Two Scikit model  2021-07-16T16:57:10.002Z  scikit-learn_0.23
569f90bf-a5f0-4f9b-9d8c-89a881493c1f  Take Two Scikit model  2021-07-16T16:41:17.002Z  scikit-learn_0.23
032f80f3-75a0-4a74-95eb-e5994465cc3f  Take Two Scikit model  2021-07-16T02:44:40.002Z  scikit-learn_0.23
------------------------------------  ---------------------  ------------------------  -----------------


In [174]:
client.deployments.list()
# use the first GUID in the scoring part

------------------------------------  -------------------  -----  ------------------------
GUID                                  NAME                 STATE  CREATED
af0cf9ba-97ba-470c-982e-665259df01bd  take-two-deployment  ready  2021-07-16T16:57:29.769Z
11633376-4d22-4934-a7ec-6e5cb3a23293  take-two-deployment  ready  2021-07-16T02:45:45.117Z
------------------------------------  -------------------  -----  ------------------------


In [286]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "take-two-deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'd195ce8b-182f-44e1-a424-817bef2896a4' started

#######################################################################################


initializing
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='af0cf9ba-97ba-470c-982e-665259df01bd'
------------------------------------------------------------------------------------------------




In [136]:
deployment_uid = client.deployments.get_uid(created_deployment)
deployment_uid

NameError: name 'created_deployment' is not defined

In [ ]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

In [249]:
client.deployments.get_details(deployment_uid)


{'entity': {'asset': {'id': '032f80f3-75a0-4a74-95eb-e5994465cc3f'},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'S', 'num_nodes': 1},
  'name': 'take-two-deployment',
  'online': {},
  'space_id': '2d3e8ea7-efa2-4efd-ae26-8a46e7a714df',
  'status': {'online_url': {'url': 'https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/11633376-4d22-4934-a7ec-6e5cb3a23293/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2021-07-16T02:45:45.117Z',
  'id': '11633376-4d22-4934-a7ec-6e5cb3a23293',
  'modified_at': '2021-07-16T02:45:45.117Z',
  'name': 'take-two-deployment',
  'owner': 'IBMid-270006CE7D',
  'space_id': '2d3e8ea7-efa2-4efd-ae26-8a46e7a714df'}}

In [121]:
marker_test.data[0]

'haha you guys are a bunch of losers.'

In [168]:
# trying to create input format that is needed for WML
count_vect = trained_model_NB.count_vect
tf_transformer = trained_model_NB.tf_transformer
clf = trained_model_NB.clf
X_new_counts = count_vect.transform(marker_test.data[0:1])
X_new_tfidf = tf_transformer.transform(X_new_counts)


valuesList = X_new_tfidf.todense().tolist()
payload_scoring = {"input_data": [{"values": valuesList}]}

In [172]:
predictions = client.deployments.score('af0cf9ba-97ba-470c-982e-665259df01bd', payload_scoring)

Failure during scoring. (POST https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/af0cf9ba-97ba-470c-982e-665259df01bd/predictions?version=2020-08-01)
Status code: 400, body: {"trace": "25b016b9c31a4d6051c29cea146b9edd", "errors": [{"code": "score_processing_failure", "message": "matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 99873 is different from 99822)"}], "status_code": 400}



ApiRequestFailure: Failure during scoring. (POST https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/af0cf9ba-97ba-470c-982e-665259df01bd/predictions?version=2020-08-01)
Status code: 400, body: {"trace": "25b016b9c31a4d6051c29cea146b9edd", "errors": [{"code": "score_processing_failure", "message": "matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 99873 is different from 99822)"}], "status_code": 400}


In [169]:
# prediction with WML

import requests

# NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account.
API_KEY = wml_credentials['apikey']
token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey": API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
# payload_scoring = {"input_data": [{"values": [X_new_tfidf.toarray()]}]}

response_scoring = requests.post('https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/11633376-4d22-4934-a7ec-6e5cb3a23293/predictions?version=2021-07-16', json=payload_scoring, headers={'Authorization': 'Bearer ' + mltoken})
print("Scoring response")
print(response_scoring.json())

Scoring response
{'trace': 'c38d87820a8113ee133df3668b9af61d', 'errors': [{'code': 'score_processing_failure', 'message': 'matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 99873 is different from 99822)'}], 'status_code': 400}


In [250]:
# this command shows the input format required by the deployed model
client.deployments.ScoringMetaNames.show()

---------------------  ----  --------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
META_PROP NAME         TYPE  REQUIRED  SCHEMA
NAME                   str   N
INPUT_DATA             list  N         [{'name(optional)': 'string', 'id(optional)': 'string', 'fields(optional)': 'array[string]', 'values': 'array[array[string]]'}]
INPUT_DATA_REFERENCES  list  N         [{'id(optional)': 'string', 'name(optional)': 'string', 'type(required)': 'string', 'connection(required)': {'href(required)': 'string'}, 'location(required)': {'bucket': 'string', 'path': 'string'}, 'schema(optional)': {'id(required)': 'string', 'fields(required)': [{'name(required)': 'string', 'type(required)': 's